In [2]:
using Pkg
Pkg.activate(".")

  Activating new environment at `~/Documents/Github2/Physics-215-Julia/Session 1 - HPC and the Julia Framework/Project.toml`


### KR1: Use ```@code_*``` to examine a simple function. The `*` is replaceable by `native`, `typed`, `warntype`, and others. Discover them.

Let us first define a simple function. Written below is a function that takes two variables and outputs their sum.

In [3]:
function f(x,y) x + y end

f (generic function with 1 method)

In [4]:
@code_native f(1,2)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[3]:1 within `f'
; │┌ @ int.jl:87 within `+'
	leaq	(%rdi,%rsi), %rax
; │└
	retq
	nopw	%cs:(%rax,%rax)
; └


In [5]:
@code_typed f(1.1,2)

CodeInfo(
1 ─ %1 = Base.sitofp(Float64, y)::Float64
│   %2 = Base.add_float(x, %1)::Float64
└──      return %2
) => Float64

In [6]:
@code_warntype f(1.1,2)

Variables
  #self#::Core.Const(f)
  x::Float64
  y::Int64

Body::Float64
1 ─ %1 = (x + y)::Float64
└──      return %1


In [7]:
@code_lowered f(1,2)

CodeInfo(
1 ─ %1 = x + y
└──      return %1
)

In [8]:
@code_llvm f(1,2)

;  @ In[3]:1 within `f'
define i64 @julia_f_1908(i64 signext %0, i64 signext %1) {
top:
; ┌ @ int.jl:87 within `+'
   %2 = add i64 %1, %0
; └
  ret i64 %2
}


Now all of these `macro` functions are new to me so I will just to explain what I understood from our class discussion and some basic googling. These `macro` functions simply convert your `code` into another form of `code` that has a different expression that I believe is more readable by the machine. Using these `@code_*` macros also show us for example, the data types of our inputs, the step-by-step process inside the function and what the output is. In Julia, there are 5 `@code_*` marcos which are `native`, `typed`, `warntype`, `lowered`, and `llvm`. These difference macro functions are I believe just different formms of how the code is being implemented by Julia.

It is also important for us to at least know how to implement these macros for cases when we are unsure of the data types of the processes that are happening inside a function. We can see for example in the code blocks above that the `code_*` marcos return `i64` and `float64` if our inputs are integers or floats for example. It also tells us the process that is being done. For example, in ` %2 = add i64 %1, %0`, we see that the function adds out inputs that are stored or located in `%1` and `%0`.

### KR2: Demonstrate that Julia is able to determine constants in codes.

Let us create a new function `g(x)` that takes a single number then performs a sequence of multiplication and division

In [42]:
function g(x)
    a = sqrt(5) * x
    return exp(2) / a
end

g (generic function with 1 method)

In [43]:
@code_llvm  g(1.1)

;  @ In[42]:1 within `g'
define double @julia_g_2003(double %0) {
top:
;  @ In[42]:2 within `g'
; ┌ @ float.jl:332 within `*'
   %1 = fmul double %0, 0x4001E3779B97F4A8
; └
;  @ In[42]:3 within `g'
; ┌ @ float.jl:335 within `/'
   %2 = fdiv double 0x401D8E64B8D4DDAE, %1
; └
  ret double %2
}


Now the important part of the result of our `@code_llvm` are the following:
```
%1 = fmul double %0, 0x4001E3779B97F4A8
%2 = fdiv double 0x401D8E64B8D4DDAE, %1
```
Now we see that `%1` represents the first line of our function which is `sqrt(5) * x`. We also see that the constant `sqrt(5)` has been stored in a memory location with value `0x4001E3779B97F4A8`. Now what Julia does is if it sees an expression that does not have a variable where it can be simplified into a single constant value, that value will be stored somewhere so that every time that specific value is needed, Julia will just pull the value from the storage instead of computing it again. This can also be seen in the next line where the value of `exp(2)` is now represented by `0x401D8E64B8D4DDAE`.

### KR3: Demonstrate Julia's type-inference and multiple dispatch.

Now before we start creating functions, we must check first if there is already an exisiting Julia function that uses the same name. For example, if we want to create a named `measure` we must first check if it is already existing. We can do this by using the help command in Julia.

In [44]:
?measure

search:

Couldn't find measure
Perhaps you meant merge


No documentation found.

Binding `measure` does not exist.


We see that there are no built-in Julia functions that have the name measure. We also see that Julia gives us possible suggestions of functions that are similar to what we searched. Let us say for example, we plan on making a function with name `exp`.

In [48]:
?log

search: log log2 log1p log10 Clong Clonglong Culong Culonglong



```
log(b,x)
```

Compute the base `b` logarithm of `x`. Throws [`DomainError`](@ref) for negative [`Real`](@ref) arguments.

# Examples

```jldoctest; filter = r"Stacktrace:(\n \[[0-9]+\].*)*"
julia> log(4,8)
1.5

julia> log(4,2)
0.5

julia> log(-2, 3)
ERROR: DomainError with -2.0:
log will only return a complex result if called with a complex argument. Try log(Complex(x)).
Stacktrace:
 [1] throw_complex_domainerror(::Symbol, ::Float64) at ./math.jl:31
[...]

julia> log(2, -3)
ERROR: DomainError with -3.0:
log will only return a complex result if called with a complex argument. Try log(Complex(x)).
Stacktrace:
 [1] throw_complex_domainerror(::Symbol, ::Float64) at ./math.jl:31
[...]
```

!!! note
    If `b` is a power of 2 or 10, [`log2`](@ref) or [`log10`](@ref) should be used, as these will typically be faster and more accurate. For example,

    ```jldoctest
    julia> log(100,1000000)
    2.9999999999999996

    julia> log10(1000000)/2
    3.0
    ```


---

```
log(x)
```

Compute the natural logarithm of `x`. Throws [`DomainError`](@ref) for negative [`Real`](@ref) arguments. Use complex negative arguments to obtain complex results.

# Examples

```jldoctest; filter = r"Stacktrace:(\n \[[0-9]+\].*)*"
julia> log(2)
0.6931471805599453

julia> log(-3)
ERROR: DomainError with -3.0:
log will only return a complex result if called with a complex argument. Try log(Complex(x)).
Stacktrace:
 [1] throw_complex_domainerror(::Symbol, ::Float64) at ./math.jl:31
[...]
```

---

```
log(A{T}::StridedMatrix{T})
```

If `A` has no negative real eigenvalue, compute the principal matrix logarithm of `A`, i.e. the unique matrix $X$ such that $e^X = A$ and $-\pi < Im(\lambda) < \pi$ for all the eigenvalues $\lambda$ of $X$. If `A` has nonpositive eigenvalues, a nonprincipal matrix function is returned whenever possible.

If `A` is symmetric or Hermitian, its eigendecomposition ([`eigen`](@ref)) is used, if `A` is triangular an improved version of the inverse scaling and squaring method is employed (see [^AH12] and [^AHR13]). For general matrices, the complex Schur form ([`schur`](@ref)) is computed and the triangular algorithm is used on the triangular factor.

[^AH12]: Awad H. Al-Mohy and Nicholas J. Higham, "Improved inverse  scaling and squaring algorithms for the matrix logarithm", SIAM Journal on Scientific Computing, 34(4), 2012, C153-C169. [doi:10.1137/110852553](https://doi.org/10.1137/110852553)

[^AHR13]: Awad H. Al-Mohy, Nicholas J. Higham and Samuel D. Relton, "Computing the Fréchet derivative of the matrix logarithm and estimating the condition number", SIAM Journal on Scientific Computing, 35(4), 2013, C394-C410. [doi:10.1137/120885991](https://doi.org/10.1137/120885991)

# Examples

```jldoctest
julia> A = Matrix(2.7182818*I, 2, 2)
2×2 Matrix{Float64}:
 2.71828  0.0
 0.0      2.71828

julia> log(A)
2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0
```
